In [21]:
import pandas as pd
from sklearn import tree
from sklearn import metrics

In [13]:
cleanedData = pd.read_csv('./Fully-Cleaned-Data.csv')

In [15]:
categoricalColumns = cleanedData.select_dtypes(include=['object']).columns.tolist()
numericalData = cleanedData.drop(columns=categoricalColumns)
oneHotData = pd.get_dummies(cleanedData[categoricalColumns])
print(numericalData.shape)
print(numericalData.columns.values)
print(oneHotData.shape)
print(oneHotData.columns.values)

(2047, 19)
['AgeatFV' 'DiseasedurationatFV' 'EDSS_FV' 'PYRAMIDAL_FUNCTION'
 'CEREBELLAR_FUNCTION' 'BRAINSTEM_FUNCTION' 'SENSORY_FUNCTION'
 'BOWEL_BLADDER_FUNCTION' 'VISUAL_FUNCTION' 'MENTAL_FUNCTION'
 'TotalnumberofrelapsesbeforeFV' 'Numberofrelapsesinthe3yearsbeforeFV'
 'Numberofrelapsesinthe1yearbeforeFV' 'timeSinceLastAttack'
 'Future Relapse Binary' 'TreatmentBeforeFV' 'RelapseInYearBeforeFVBinary'
 'RelapseInThe3YearsBeforeFVBinary' 'Treatment with Injectable Med']
(2047, 29)
['SEX_F' 'SEX_M' 'RACE_DESC_American Indian or Alaska Native'
 'RACE_DESC_Asian' 'RACE_DESC_Black or African American'
 'RACE_DESC_East Asian and South-East' 'RACE_DESC_More than one race'
 'RACE_DESC_Native Hawaiian or Other Pacific Islander'
 'RACE_DESC_South Asian' 'RACE_DESC_Unknown or not reported'
 'RACE_DESC_White' 'ETHNICITY_DESC_Hispanic or latino'
 'ETHNICITY_DESC_Not hispanic or latino' 'ETHNICITY_DESC_Unknown'
 'FAMILY_MS_N' 'FAMILY_MS_U' 'FAMILY_MS_Y' 'SMOKING_EVER_N'
 'SMOKING_EVER_Y' 'DISEASE_C

In [20]:
cleanedDataOneHotEncoded = pd.concat([numericalData, oneHotData], axis=1)
print(cleanedDataOneHotEncoded.shape)

(2047, 48)


In [ ]:
import warnings
warnings.filterwarnings('ignore')

treeClassifier = tree.DecisionTreeClassifier(max_depth=5)
indicator = cleanedDataOneHotEncoded['Future Relapse Binary']
X = cleanedDataOneHotEncoded.drop(['Future Relapse Binary'], axis=1)
treeModel = treeClassifier.fit(X, indicator)

classificationResults = pd.DataFrame(columns=['Real Label', 'Predicted Label', 'Difference'])
classificationResultsArray = []

for _, row in cleanedDataOneHotEncoded.iterrows():
    classificationResultsArray.append(treeModel.predict([row[1:]])[0])

classificationResults['Predicted Label'] = pd.Series(classificationResultsArray)
classificationResults['Real Label'] = cleanedDataOneHotEncoded['Future Relapse Binary']
classificationResults['Difference'] = abs(classificationResults['Real Label'] - classificationResults['Predicted Label'])
diffValueCounts = classificationResults['Difference'].value_counts()
print(diffValueCounts)
print(f'Accuracy: {(diffValueCounts[0]/(diffValueCounts[0] + diffValueCounts[1]))*100:.2f}%')

Difference
0    1230
1     817
Name: count, dtype: int64
Accuracy: 60.09%


In [62]:
precScore = metrics.precision_score(cleanedDataOneHotEncoded['Future Relapse Binary'], classificationResults['Predicted Label'])
recScore = metrics.recall_score(cleanedDataOneHotEncoded['Future Relapse Binary'], classificationResults['Predicted Label'])
print(precScore)
print(recScore)

0.40093240093240096
0.23497267759562843
